In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model

#load separated txt datasets
deaths01 = pd.read_csv("../data_raw/mortality_ages_0-10.txt", sep = "\t", encoding = "ISO-8859-1")
deaths02 = pd.read_csv("../data_raw/mortality_ages_11-15.txt", sep = "\t", encoding = "ISO-8859-1")
deaths03 = pd.read_csv("../data_raw/mortality_ages_16-20.txt", sep = "\t", encoding = "ISO-8859-1")
deaths04 = pd.read_csv("../data_raw/mortality_ages_21-25.txt", sep = "\t", encoding = "ISO-8859-1")
deaths05 = pd.read_csv("../data_raw/mortality_ages_26-30.txt", sep = "\t", encoding = "ISO-8859-1")
deaths06 = pd.read_csv("../data_raw/mortality_ages_31-35.txt", sep = "\t", encoding = "ISO-8859-1")
deaths07 = pd.read_csv("../data_raw/mortality_ages_36-40.txt", sep = "\t", encoding = "ISO-8859-1")
deaths08 = pd.read_csv("../data_raw/mortality_ages_41-45.txt", sep = "\t", encoding = "ISO-8859-1")
deaths09 = pd.read_csv("../data_raw/mortality_ages_46-50.txt", sep = "\t", encoding = "ISO-8859-1")
deaths10 = pd.read_csv("../data_raw/mortality_ages_51-55.txt", sep = "\t", encoding = "ISO-8859-1")
deaths11 = pd.read_csv("../data_raw/mortality_ages_56-60.txt", sep = "\t", encoding = "ISO-8859-1")
deaths12 = pd.read_csv("../data_raw/mortality_ages_61-65.txt", sep = "\t", encoding = "ISO-8859-1")
deaths13 = pd.read_csv("../data_raw/mortality_ages_66-70.txt", sep = "\t", encoding = "ISO-8859-1")
deaths14 = pd.read_csv("../data_raw/mortality_ages_71-75.txt", sep = "\t", encoding = "ISO-8859-1")
deaths15 = pd.read_csv("../data_raw/mortality_ages_76-80.txt", sep = "\t", encoding = "ISO-8859-1")
deaths16 = pd.read_csv("../data_raw/mortality_ages_81-85.txt", sep = "\t", encoding = "ISO-8859-1")
deaths17 = pd.read_csv("../data_raw/mortality_ages_86-90.txt", sep = "\t", encoding = "ISO-8859-1")
deaths18 = pd.read_csv("../data_raw/mortality_ages_91-95.txt", sep = "\t", encoding = "ISO-8859-1")
deaths19 = pd.read_csv("../data_raw/mortality_ages_96-100_plus.txt", sep = "\t", encoding = "ISO-8859-1")

#put together dataset
deaths = pd.concat([deaths01, deaths02, deaths03, deaths04, deaths05, deaths06, deaths07, deaths08, deaths09,
                   deaths10, deaths11, deaths12, deaths13, deaths14, deaths15, deaths16, deaths17, deaths18,
                   deaths19])

#filter dataset by columns
deaths = deaths[['Single-Year Ages Code', 'Gender', 'Race', 'Injury Mechanism & All Other Leading Causes',
                 'Cause of death', 'Deaths', 'Population']]

#rename columns 
deaths.columns = ['age','gender','race','mechanism_of_death','cause_of_death','deaths','population']

#replace Not Applicable with NaN null values
deaths = deaths.replace('Not Applicable', np.NaN)

#convert population to float
deaths['population'] = deaths['population'].astype('float')

#collect extrapolated population numbers
pred_dfs = []
for race in deaths.race.unique():
    for sex in deaths.gender.unique():
        temp_df = deaths[(deaths.race == race)&(deaths.gender == sex)&(deaths.age >= 70)][['age','population']].drop_duplicates()
        reg = linear_model.LinearRegression()
        reg.fit(X=np.array(temp_df[temp_df.age < 85]['age']).reshape(-1,1), y = np.array(temp_df[temp_df.age < 85]['population']))
        y_pred = np.clip(reg.predict(np.array(temp_df[temp_df.age >= 85]['age']).reshape(-1,1)), a_min = 0, a_max = float('inf')).round()
        x_pred = range(85,101)
        pred_df = pd.DataFrame({'age':x_pred,'pred_population':y_pred, 'race':race, 'gender':sex})
        pred_dfs.append(pred_df)
pred_df_full = pd.concat(pred_dfs)

#load extrapolated dataset and replace with null population values
deaths = deaths.merge(pred_df_full, left_on=['age', 'gender', 'race'], right_on=['age', 'gender', 'race'], how = 'left')
deaths['population']=deaths['population'].mask(pd.isnull, deaths['pred_population'])
deaths = deaths.drop('pred_population', axis=1)